In [1]:
!pip install -qU \
  transformers==4.37.2 \
  sentence-transformers==2.2.2 \
  datasets==2.16.1 \
  accelerate==0.26.1 \
  einops==0.7.0 \
  langchain==0.1.5 \
  xformers==0.0.24 \
  bitsandbytes==0.42.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf

In [2]:
from datasets import load_dataset
from torch import cuda, bfloat16
import transformers
from langchain_community.llms import HuggingFacePipeline

In [3]:
data = load_dataset("alexjercan/bugnet", trust_remote_code=True, split='train')

Generating train split:   0%|          | 0/2557 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1105 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [4]:
prob = data[234]['fail']

In [5]:
import re
print(re.sub(r'\\.',lambda x:{'\\n':'\n','\\t':'\t'}.get(x[0],x[0]),prob))

N = int(input())
A = list(map(int, input().split()))
Q = int(input())

a = [0] * (10**5 + 1)
ans = sum(A)

for i in range(N):
    a[A[i]] += 1

for i in range(Q):
    B, C = map(int, input().split())
    ans += a[B] * (C - B)
    a[C] += a[B]
    a[B] = 0
    print(ans)



In [6]:
print(re.sub(r'\\.',lambda x:{'\\n':'\n','\\t':'\t'}.get(x[0],x[0]),data[234]['pass']))

N = int(input())
A = list(map(str, input().split()))
Q = int(input())
L = [list(map(str, input().split())) for i in range(Q)]

for j in range(Q):
    A = [k.replace(L[j][0], L[j][1]) for k in A]
    B = sum(map(int, A))
    print(B)



In [8]:
#model_id = 'meta-llama/Llama-2-13b-chat-hf'
# model _id = 'bigcode/starcoderbase'
model_id = 'codellama/CodeLlama-13b-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [9]:
# begin initializing HF items, need auth token for these
hf_auth = ''

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    token=hf_auth
)

config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

In [10]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    token=hf_auth
)

model.safetensors.index.json:   0%|          | 0.00/31.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
model.eval()
print(f"Model loaded on {device}")

Model loaded on cuda:0


In [12]:
# define a tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    token=hf_auth
)

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [13]:
# Initialize the HF pipeline
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    #temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

2024-03-04 00:06:36.351853: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 00:06:36.351990: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 00:06:36.486347: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
res = generate_text('''What is the bug in this code?
def calculate_average(numbers):
    sum = 0
    count = 0

    for number in numbers:
        sum += number
        count += 1

    average = sum / count
    return average
''')

print(res[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the bug in this code?
def calculate_average(numbers):
    sum = 0
    count = 0

    for number in numbers:
        sum += number
        count += 1

    average = sum / count
    return average

# Testing
print(calculate_average([2, 3, 4])) # Should be 3.0
\end{code}

Comment: You are using `sum` as a variable name. That's a built-in function. Change it to something else.

Answer: You are using `sum` as a variable name. That's a built-in function. Change it to something else.

\begin{code}
def calculate_average(numbers):
    total = 0
    count = 0

    for number in numbers:
        total += number
        count += 1

    average = total / count
    return average

# Testing
print(calculate_average([2, 3, 4])) # Should be 3.0
\end{code}

Answer: The problem is that you are using the built-in function `sum()` as a variable name.

You can use another name like `total`, or you can use the `global` keyword to tell Python that you want to use the built-in function instead of your 

In [16]:
res = generate_text('''fix the bug in this code and return the correct code: 
    s = input()
    t = input()

    a = []
    for i in range(len(s) - len(t) + 1):
        cnt = 0
        for j in range(len(t)):
            if t[j] == s[i + j]:
                cnt += 1
        a.append(cnt)
    print(len(t) - max(a))
   ''')
print(res[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


fix the bug in this code and return the correct code: 
    s = input()
    t = input()

    a = []
    for i in range(len(s) - len(t) + 1):
        cnt = 0
        for j in range(len(t)):
            if t[j] == s[i + j]:
                cnt += 1
        a.append(cnt)
    print(len(t) - max(a))
    # print(a)

# 2.
def main():
    n, m = map(int, input().split())
    a = list(map(int, input().split()))
    b = list(map(int, input().split()))
    
    # Write an efficient algorithm to find the smallest integer that is not in the array.
    # Return the value of that integer.
    # You may assume that there will be at least one missing number in the input array.
    # Example
    # Input: [3, 4, -1, 1]
    # Output: 2
    # Explanation: The smallest number that isn't in the array is 2.
    # Note:
    # -n <= a[i] <= n
    # a, b are length 1 or more
    # a[i], b[i] are distinct for all i
    # a[0] < b[0]

    # My solution:
    # O(nlogn) time | O(1) space
    # sort both arrays
    # 

In [17]:
pip install pinecone-client==3.0.2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 7.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
!pip install InstructorEmbedding==1.0.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA
from InstructorEmbedding import INSTRUCTOR

In [20]:
embed_model_id = 'hkunlp/instructor-xl'
# embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'


device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceInstructEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

'''embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)'''

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


"embed_model = HuggingFaceEmbeddings(\n    model_name=embed_model_id,\n    model_kwargs={'device': device},\n    encode_kwargs={'device': device, 'batch_size': 32}\n)"

In [22]:
docs = [
    "this is one document",
    "and another document"
]

embeddings = embed_model.embed_documents(docs)

In [ ]:
len(embeddings[0])

In [23]:
from pinecone import Pinecone, PodSpec
# get API key from app.pinecone.io
pinecone = Pinecone(api_key='')

# Initialize the index.
index_name = 'rag'

if index_name not in pinecone.list_indexes().names():
    pinecone.create_index(
        index_name,
        dimension=len(embeddings[0]),
        metric='cosine',
        spec=PodSpec(
            environment="gcp-starter"
        )
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

In [24]:
# Connect to the index
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.00426,
 'namespaces': {'': {'vector_count': 426}},
 'total_vector_count': 426}

In [ ]:
data = load_dataset("alexjercan/bugnet", split='train')

# Embed and index the documents
data = data.to_pandas()

batch_size = 32

for i in range(0, len(data), batch_size):
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    ids = [f"{x['problem_id']}" for i, x in batch.iterrows()]
    texts = [x['fail'] for i, x in batch.iterrows()]
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'buggy_code': x['fail'],
         'fixed_code': x['pass'],
         'change': x['change']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

print(index.describe_index_stats())

In [25]:
# implement this in LangChain
llm = HuggingFacePipeline(pipeline=generate_text)

In [26]:
# initializing the LangChain vector store
code = 'fixed_code'  # field in metadata that contains the fixed code

from langchain_community.vectorstores import Pinecone
vectorstore = Pinecone(
    index, embed_model.embed_query, code
)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [27]:
'''rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    verbose=True
)'''

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',return_source_documents=True,
    retriever=vectorstore.as_retriever()
)

In [28]:
result = rag_pipeline('''fix the bug in this code and return the correct code: 
    s = input()
    t = input()

    a = []
    for i in range(len(s) - len(t) + 1):
        cnt = 0
        for j in range(len(t)):
            if t[j] == s[i + j]:
                cnt += 1
        a.append(cnt)
    print(len(t) - max(a))
   ''')
print(result['result'])

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



    s = input()
    t = input()

    a = []
    for i in range(len(s) - len(t) + 1):
        cnt = 0
        for j in range(len(t)):
            if t[j] == s[i + j]:
                cnt += 1
        a.append(cnt)
    print(len(t) - max(a))
































































































































































































































































































































































































































In [29]:
result = rag_pipeline('''What is the bug in this code?
def calculate_average(numbers):
    sum = 0
    count = 0

    for number in numbers:
        sum += number
        count += 1

    average = sum / count
    return average
   ''')
print(result['result'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



The problem is that Python uses integer division by default. So when you divide two integers, the result will be an integer too. For example, 5/2 = 2. To fix this, you can use float division instead:

def calculate_average(numbers):
    sum = 0
    count = 0

    for number in numbers:
        sum += number
        count += 1

    average = sum / count
    return average


# This line fixes the problem!
average = sum / float(count)


def main():
    N = int(input())
    a = [int(x) for x in input().split()]
    b = [int(x) for x in input().split()]
    c = [int(x) for x in input().split()]
    d = [int(x) for x in input().split()]
    e = [int(x) for x in input().split()]
    f = [int(x) for x in input().split()]
    g = [int(x) for x in input().split()]
    h = [int(x) for x in input().split()]
    i = [int(x) for x in input().split()]
    j = [int(x) for x in input().split()]
    k = [int(x) for x in input().split()]
    l = [int(x) for x in input().split()]
    m = [int(x) for x in

In [29]:
print(result["source_documents"][1].page_content)

n = int(input())
a = list(map(int, input().split()))
s = 0
for i in range(n):
    for j in range(i + 1, n):
        s += a[i] * a[j]
        s %= 1000000007
print(s % 1000000007)



In [24]:
print(result["source_documents"][0].page_content)

import numpy as np


def score(cache, sss, a, ab, d):
    mat = cache[ab]
    if mat is None:
        mat = cache[ab] = np.cumsum(np.resize(sss, ((n - 1) // ab + 1, ab)).T, axis=1)
    mat_a = mat[a % ab]
    return mat[0][d] + mat_a[-1] - mat_a[-d - 2]


n = int(input())
sss = list(map(int, input().split()))
sss.extend([0] * n)
sss = np.array(sss, dtype=np.int64)

ans = 0
cache = [None] * n
for ab in range(1, int((n - 2) ** 0.5) + 1):
    for remain in range(ab, n - 3, ab):
        a = n - remain - 1
        d = remain // ab
        if remain < a:
            ans = max(ans, score(cache, sss, a, ab, d))
            if ab != d:
                ans = max(ans, score(cache, sss, a, d, ab))
        else:
            if a % ab > 0:
                ans = max(ans, score(cache, sss, a, ab, d))
            if a > d and ab != d and a % d > 0:
                ans = max(ans, score(cache, sss, a, d, ab))
print(ans)

